# Manual Testing

In [2]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

### Test Ensure

In [7]:
compileString("""
procedure fact(n: integer) → (f: integer)
    ensure f ≥ n
    if n = 0 then f := 1
    else
        f ← fact(n - 1); f := f × n

program factorial
    var y, z: integer
        y ← read(); z ← fact(y); write(z)
        
""", 'testensure.wat')

In [4]:
# !type testensure.wat

In [5]:
!wsl.exe wat2wasm testensure.wat

In [6]:
runwasm("testensure.wasm")

<IPython.core.display.Javascript object>

In [3]:
compileString("""
procedure fact(n: integer) → (f: integer)
    ensure n < 5
    if n = 0 then f := 1
    else
        f ← fact(n - 1); f := f × n

program factorial
    var y, z: integer
        y ← read(); z ← fact(y); write(z)
        
""", 'testensure2.wat')

['local.get $n', 'i32.const 5', 'i32.lt_s']
Var(name = , lev = -1, tp = <class 'ST.Bool'>)


Exception: 'tuple' object has no attribute 'append'

In [23]:
# !type testensure2.wat

In [21]:
!wsl.exe wat2wasm testensure2.wat

In [22]:
runwasm("testensure2.wasm")

<IPython.core.display.Javascript object>

### Test Invariant

In [83]:
compileString("""
procedure proc() → (q: integer)
    var x: integer
        q := 5
        x := 3
        invariant x > 1
        while x ≥ 0 do
            q := q + 1
            x := x - 1
program test
    var j: integer
    j ← proc()
    write(j)
""", 'test2.wat')

In [3]:
!cat test2.wat

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $proc  (result i32)
(local $q i32)
(local $x i32)
(local $0 i32)
i32.const 5
local.set $q
i32.const 3
local.set $x
loop
local.get $x
i32.const 0
i32.ge_s
if
local.get $x
i32.const 1
i32.gt_s
if
local.get $q
i32.const 1
i32.add
local.set $q
local.get $x
i32.const 1
i32.sub
local.set $x
else
unreachable
end
br 1
end
end
local.get $q)
(func $program
(local $j i32)
(local $0 i32)
call $proc
local.set $j
local.get $j
call $write
)
(memory 1)
(start $program)
)

In [5]:
!wat2wasm test2.wat

In [6]:
runwasm("test2.wasm")

<IPython.core.display.Javascript object>

In [7]:
compileString("""
procedure proc() → (q: integer)
    var x: integer
        q := 5
        x := 3
        require x > 0
        while x ≥ 0 do
            q := q + 1
            x := x - 1
program test
    var j: integer
    j ← proc()
    write(j)
""", 'test2.wat')

In [22]:
!wat2wasm test2.wat

In [23]:
runwasm("test2.wasm")

<IPython.core.display.Javascript object>

# Test Require

In [ ]:
# Test: proper usage example 1
compileString("""
procedure reqhere (x: integer) → (q : integer)
    require (x > 5) and (x <7) 
    q := x
    write(x)

program test
  var x , y: integer
    x ← read()
    y ← reqhere(x)
    write(y)
""", 'require_test1.wat')

In [ ]:
!wat2wasm require_test1.wat
runwasm("require_test1.wasm")

In [ ]:
# Test: proper usage example 2 
compileString("""
procedure reqhere (x, y: integer) → (q : integer)
    require (x > 5) or (y <7) 
    q := x
    write(x)

program test
  var x , z: integer
    x ← read()
    y ← read()
    z ← reqhere(x)
    write(z)
""", 'require_test2.wat')

In [ ]:
!wat2wasm require_test2.wat
runwasm("require_test2.wasm")

In [ ]:
# Test: improper usage example 1
compileString("""
procedure reqhere (x, y: integer) → (q : integer)
    q := x
    require (x > 5) or (y <7) 
    write(x)

program test
  var x , z: integer
    x ← read()
    y ← read()
    z ← reqhere(x)
    write(z)
""", 'require_test3.wat')

In [ ]:
# should return error
!wat2wasm require_test3.wat
runwasm("require_test3.wasm")

In [ ]:
# Test: improper usage example 2
compileString("""
procedure reqhere (x, y: integer) → (q : integer)
    q := x
    write(x)

program test
  var x , z: integer
    x ← read()
    y ← read()
    require (x > 5) or (y <7) 
    z ← reqhere(x)
    write(z)
""", 'require_test4.wat')

In [ ]:
# should return error
!wat2wasm require_test4.wat
runwasm("require_test4.wasm")